<a href="https://colab.research.google.com/github/haeseokoh/00_exchange/blob/main/%EC%83%81%EB%8C%80%EA%B0%95%EB%8F%84_%EA%B0%95%ED%95%9C_%EB%84%98%EB%93%A4%EB%A1%9C_%ED%88%AC%EC%9E%90(%EB%AF%B8%EA%B5%AD)%EC%A2%85%EB%AA%A9%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. 날짜 입력받기
target_date = input("원하는 날짜를 입력하세요 (예: 2025-04-05): ")

# 2. 데이터 다운로드
!pip install yfinance --quiet
import yfinance as yf
import pandas as pd

extra_tickers = ['QQQ', 'SPY', 'PSQ', 'IAU', 'BIL', 'SPMO']

data = yf.download(extra_tickers, period="2y", interval="1d", auto_adjust=False)['Close']

# 3. 가장 가까운 거래일 찾기
data = data.dropna(how='all')  # 완전 비어있는 날 제거
available_dates = data.index

target_timestamp = pd.to_datetime(target_date)

# 입력 날짜 이전의 가장 가까운 거래일 찾기
available_dates_before = available_dates[available_dates <= target_timestamp]
if len(available_dates_before) == 0:
    raise ValueError("입력한 날짜 이전에 데이터가 없습니다.")

real_date = available_dates_before[-1]  # 가장 최근 거래일

print(f"\n📅 실제 사용 날짜: {real_date.date()}")

# 4. 이동평균 계산
qqq_prices = data['QQQ']
qqq_ma10 = qqq_prices.rolling(window=10).mean()
qqq_ma20 = qqq_prices.rolling(window=20).mean()

spmo_prices = data['SPMO']
spmo_ma10 = spmo_prices.rolling(window=10).mean()
spmo_ma20 = spmo_prices.rolling(window=20).mean()

# 5. 카나리아 신호 계산
qqq_canary_ok = (
    (qqq_prices.loc[real_date] > qqq_ma10.loc[real_date]) or
    (qqq_prices.loc[real_date] > qqq_ma20.loc[real_date])
)

spmo_canary_ok = (
    (spmo_prices.loc[real_date] > spmo_ma10.loc[real_date]) or
    (spmo_prices.loc[real_date] > spmo_ma20.loc[real_date])
)

# 6. NASDAQ100 + S&P100 티커 가져오기
def get_nasdaq100_tickers():
    url = 'https://en.wikipedia.org/wiki/NASDAQ-100'
    tables = pd.read_html(url)
    table = tables[4]
    tickers = table['Ticker'].tolist()
    tickers = [ticker.replace('.', '-') for ticker in tickers]
    return tickers

def get_sp100_tickers():
    url = 'https://en.wikipedia.org/wiki/S%26P_100'
    tables = pd.read_html(url)
    table = tables[2]
    tickers = table['Symbol'].tolist()
    tickers = [ticker.replace('.', '-') for ticker in tickers]
    return tickers

nasdaq100 = get_nasdaq100_tickers()
sp100 = get_sp100_tickers()
combined_tickers = list(set(nasdaq100 + sp100))

# 7. NASDAQ100 + S&P100 데이터 추가 다운로드
stock_data = yf.download(combined_tickers, period="2y", interval="1d", auto_adjust=False)['Close']
stock_data = stock_data.dropna(how='all', axis=1)

# 8. 모멘텀 점수 계산 (3M, 6M, 12M 수익률)
returns_3m = stock_data.pct_change(63)
returns_6m = stock_data.pct_change(126)
returns_12m = stock_data.pct_change(252)
weighted_momentum = (returns_12m * 4) + (returns_6m * 3) + (returns_3m * 2)

momentum_today = weighted_momentum.loc[real_date].dropna()

# 9. 상위 2개 모멘텀 종목 추출
top2 = momentum_today.sort_values(ascending=False).head(2).index.tolist()

# 10. 포트폴리오 추천
print("\n✅ 추천 포트폴리오:")
if qqq_canary_ok:
    if not spmo_canary_ok:
        print(f"✅ QQQ 70%, SPY 10%, IAU 10%, BIL 10%")
    else:
        print(f"✅ {', '.join(top2)}, SPY, IAU, BIL (Top2 each 35%, SPY 10%, IAU 10%, BIL 10%)")
else:
    print(f"✅ PSQ 50%, IAU 10%, BIL 10%, {', '.join(top2)} (Top2 each 15%)")


원하는 날짜를 입력하세요 (예: 2025-04-05): 2025-05-01


[*********************100%***********************]  6 of 6 completed



📅 실제 사용 날짜: 2025-04-30


[*********************100%***********************]  170 of 170 completed



✅ 추천 포트폴리오:
✅ PLTR, APP, SPY, IAU, BIL (Top2 each 35%, SPY 10%, IAU 10%, BIL 10%)
